In [2]:
# Implement GoogleNet
# Build an inception module
%pip install sqlalchemy

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [3]:
import functools
import keras

DefaultConv = functools.partial(
    keras.layers.Conv2D, kernel_size=(1, 1), strides=(1, 1),
    padding="same", activation="relu")

class InceptionModule(keras.layers.Layer):
  def __init__(self, filters11, filters33_reduce, filters33,
               filters55_reduce, filters55, filters_pool_proj,
               use_batch_norm=True,**kwargs):
    super().__init__(**kwargs)
    self.conv11 = DefaultConv(filters=filters11)

    self.conv33_reduce = DefaultConv(filters=filters33_reduce)
    self.conv33 = DefaultConv(filters=filters33, kernel_size=(3,3))

    self.conv55_reduce = DefaultConv(filters=filters55_reduce)
    self.conv55 = DefaultConv(filters=filters55, kernel_size=(5,5))

    self.max_pool33 = keras.layers.MaxPool2D(pool_size=(3,3), strides=(1,1), padding='same')
    self.conv_pool = DefaultConv(filters=filters_pool_proj)

    self.use_batch_norm = use_batch_norm
    if use_batch_norm:
      self.batch_layer = keras.layers.BatchNormalization()

  def call(self, inputs):
    path1 = self.conv11(inputs)
    path2 = self.conv33(self.conv33_reduce(inputs))
    path3 = self.conv55(self.conv55_reduce(inputs))
    path4 = self.conv_pool(self.max_pool33(inputs))
    # print('path1', path1.shape)
    # print('path2', path2.shape)
    # print('path3', path3.shape)
    # print('path4', path4.shape)

    iModule = keras.layers.Concatenate(axis=-1)([path1, path2, path3, path4])
    # print('imodule', iModule.shape)
    logits = self.batch_layer(iModule) if self.use_batch_norm else iModule

    return logits


2025-01-07 14:22:47.653173: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-07 14:22:47.664043: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736256167.676604  268312 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736256167.680359  268312 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-07 14:22:47.700823: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
DefaultMaxPool = functools.partial(
    keras.layers.MaxPool2D,
    pool_size=(3,3), strides=(2,2), padding="same")

def get_googlenet_model(input_shape, num_classes, use_batch_norm=True, **kwargs):
  model = keras.Sequential(**kwargs)
  model.add(keras.layers.Input(shape=input_shape))
  model.add(DefaultConv(filters=64, kernel_size=(7,7), strides=(2,2),  padding='same'))
  if use_batch_norm:
    model.add(keras.layers.BatchNormalization())
  model.add(DefaultMaxPool())
  model.add(DefaultConv(filters=64))
  model.add(DefaultConv(filters=192, kernel_size=(3,3)))
  if use_batch_norm:
    model.add(keras.layers.BatchNormalization())
  model.add(DefaultMaxPool())

  model.add(InceptionModule(filters11=64, filters33_reduce=96, filters33=128,
    filters55_reduce=16, filters55=32, filters_pool_proj=32,
    use_batch_norm=use_batch_norm))
  model.add(InceptionModule(filters11=128, filters33_reduce=128, filters33=192,
    filters55_reduce=32, filters55=96, filters_pool_proj=64,
    use_batch_norm=use_batch_norm))
  model.add(DefaultMaxPool())

  model.add(InceptionModule(filters11=192, filters33_reduce=96, filters33=208,
    filters55_reduce=16, filters55=48, filters_pool_proj=64,
    use_batch_norm=use_batch_norm))
  model.add(InceptionModule(filters11=160, filters33_reduce=112, filters33=224,
    filters55_reduce=24, filters55=64, filters_pool_proj=64,
    use_batch_norm=use_batch_norm))
  model.add(InceptionModule(filters11=128, filters33_reduce=128, filters33=256,
    filters55_reduce=24, filters55=64, filters_pool_proj=64,
    use_batch_norm=use_batch_norm))
  model.add(InceptionModule(filters11=112, filters33_reduce=144, filters33=288,
    filters55_reduce=32, filters55=64, filters_pool_proj=64,
    use_batch_norm=use_batch_norm))
  model.add(InceptionModule(filters11=256, filters33_reduce=160, filters33=320,
    filters55_reduce=32, filters55=128, filters_pool_proj=128,
    use_batch_norm=use_batch_norm))

  model.add(DefaultMaxPool())
  model.add(InceptionModule(filters11=256, filters33_reduce=160, filters33=320,
    filters55_reduce=32, filters55=128, filters_pool_proj=128,
    use_batch_norm=use_batch_norm))
  model.add(InceptionModule(filters11=384, filters33_reduce=192, filters33=384,
    filters55_reduce=48, filters55=128, filters_pool_proj=128,
    use_batch_norm=use_batch_norm))
  model.add(keras.layers.GlobalAveragePooling2D())
  model.add(keras.layers.Dropout(0.4))
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(units=1000, activation="relu"))
  model.add(keras.layers.Dense(units=4, activation='sigmoid'))

  return model

In [5]:
model = get_googlenet_model(input_shape=(1024,1024,3), num_classes=4)
model.summary()

I0000 00:00:1736256171.145904  268312 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4269 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 512, 512, 64)   │         9,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512, 512, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 256, 256, 64)   │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 256, 256, 192)  │       110,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256, 256, 192)  │           768 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 128, 128, 192)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module                │ (None, 128, 128, 256)  │       164,720 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_1              │ (None, 128, 128, 480)  │       390,656 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 64, 64, 480)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_2              │ (None, 64, 64, 512)    │       378,224 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_3              │ (None, 64, 64, 512)    │       451,208 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_4              │ (None, 64, 64, 512)    │       512,152 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_5              │ (None, 64, 64, 528)    │       607,488 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_6              │ (None, 64, 64, 832)    │       871,680 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 32, 32, 832)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_7              │ (None, 32, 32, 832)    │     1,046,784 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_8              │ (None, 32, 32, 1024)   │     1,448,176 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │             

 Total params: 7,025,532 (26.80 MB)

 Trainable params: 7,014,044 (26.76 MB)

 Non-trainable params: 11,488 (44.88 KB)

In [6]:
from helpers import iou

In [7]:
# Compile the model with IoU loss
# model.compile(optimizer='adam', loss=keras.losses.Huber(), metrics=[iou_loss])
model.compile(optimizer='adam', loss='mse', metrics=[iou])


In [18]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from FrameLoader import FrameLoader
from DataGeneratorFrames import DataGeneratorFrames
from DataRepository import DataRepository

repo = DataRepository()
repo.load_relativePaths_of_videos_with_framelabels()

DIM = 320
train_generator = DataGeneratorFrames(
    frameloader=FrameLoader(repo),
    train_test_val="train",
    dim=(DIM,DIM),
    batch_size=32,
)

val_generator = DataGeneratorFrames(
    frameloader=FrameLoader(repo),
    train_test_val="test",
    dim=(DIM,DIM),
    batch_size=32,
)


callbacks = [
    ModelCheckpoint('model_best.keras', save_best_only=True, monitor='loss', mode='min', verbose=1),
    EarlyStopping(monitor='loss', patience=2, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='loss', factor=0.5, patience=1, verbose=1)
]

FIT = False

if FIT:
    history = model.fit(
        train_generator,
        epochs=7,
        callbacks=callbacks,
        verbose=1,
        validation_data=val_generator
    )
    keras.models.save_model(
        model,
        filepath="googlenet.keras",
        overwrite=True
    )

    model.save_weights("googlenet.weights.h5")

connection established
connection established
DataGeneratorSkillBorders init done
connection established
DataGeneratorSkillBorders init done


In [19]:
X, y = train_generator.__getitem__(5)
X.shape, y.shape

((32, 320, 320, 3), (32, 4))

In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 512, 512, 64)   │         9,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512, 512, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 256, 256, 64)   │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 256, 256, 192)  │       110,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256, 256, 192)  │           768 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 128, 128, 192)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module                │ (None, 128, 128, 256)  │       164,720 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_1              │ (None, 128, 128, 480)  │       390,656 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 64, 64, 480)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_2              │ (None, 64, 64, 512)    │       378,224 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_3              │ (None, 64, 64, 512)    │       451,208 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_4              │ (None, 64, 64, 512)    │       512,152 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_5              │ (None, 64, 64, 528)    │       607,488 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_6              │ (None, 64, 64, 832)    │       871,680 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 32, 32, 832)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_7              │ (None, 32, 32, 832)    │     1,046,784 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_module_8              │ (None, 32, 32, 1024)   │     1,448,176 │
│ (InceptionModule)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │             

 Total params: 7,025,532 (26.80 MB)

 Trainable params: 7,014,044 (26.76 MB)

 Non-trainable params: 11,488 (44.88 KB)

In [21]:
model.load_weights("googlenet.weights.h5")

/home/miked/code/judge/computervision/.venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 282 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [22]:
X, y = val_generator.__getitem__(2)
X.shape, y.shape

((27, 320, 320, 3), (27, 4))

In [23]:
model.predict(X)

2025-01-07 14:28:40.429178: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_845', 16 bytes spill stores, 16 bytes spill loads

2025-01-07 14:28:40.762746: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_845', 140 bytes spill stores, 140 bytes spill loads

2025-01-07 14:28:41.091327: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_845_0', 240 bytes spill stores, 240 bytes spill loads

2025-01-07 14:28:41.214176: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_852', 364 bytes spill stores, 364 bytes spill loads



1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step


array([[0.9778031 , 0.90072805, 0.01080329, 0.00119205],
       [0.9777955 , 0.9003129 , 0.01076904, 0.00120013],
       [0.97788435, 0.9005719 , 0.01138563, 0.00121879],
       [0.9779087 , 0.8996741 , 0.01081332, 0.0012066 ],
       [0.9778364 , 0.89955735, 0.01080503, 0.00120693],
       [0.97784835, 0.8993787 , 0.0108344 , 0.00121057],
       [0.9778144 , 0.9001923 , 0.01080978, 0.00119916],
       [0.9778112 , 0.89991266, 0.01087519, 0.00120431],
       [0.9778816 , 0.89933515, 0.01084105, 0.00121024],
       [0.97788984, 0.90038705, 0.01076725, 0.00119609],
       [0.97773415, 0.9012917 , 0.01140377, 0.0012171 ],
       [0.9779344 , 0.90060174, 0.01142519, 0.00121672],
       [0.9778146 , 0.8998148 , 0.01080398, 0.00120153],
       [0.9778526 , 0.89971423, 0.01080175, 0.00120415],
       [0.97787535, 0.9006072 , 0.01141744, 0.00121573],
       [0.97784895, 0.89990824, 0.01089121, 0.00120299],
       [0.97778404, 0.9000867 , 0.01083438, 0.00120089],
       [0.97782   , 0.9006151 ,